In [1]:
from bs4 import BeautifulSoup
import requests

url = "https://opencircuit.nl/list/arduino"

page = requests.get(url)

soup = BeautifulSoup(page.text, 'html')

print(soup)

 <!DOCTYPE HTML>
<html lang="nl"><head><title>Arduino kopen? - Opencircuit</title><meta charset="utf-8"/><meta content="Ontdek Arduino in onze webshop! Bouw robots, ontwerp weerstations en automatiseer je huis. Begin nu met Arduino. Snelle levering gegarandeerd!" name="description"/><meta content="index, follow" name="robots"/><meta content="2044467" name="PageID"/><meta content="E7CZrGOOesKHJwWuEG0K_D3XWacCTn_8WLSBsu8IXLQ" name="google-site-verification"><link href="/opensearch.xml" rel="search" title="Opencircuit Search" type="application/opensearchdescription+xml"/> <meta content="EUR" data-symbol="€" name="currency"/><link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/><link href="cdn.bodanius.com" rel="preconnect"/> <link href="https://opencircuit.nl/list/arduino" rel="canonical"/> <link href="https://opencircuit.nl/list/arduino" hreflang="nl" rel="alternate"/> <link href="https://opencircu.it/list/arduino" hreflang="it" rel="alternate"/> <link href="https://ope

In [68]:
table = soup.find("div", {'class':'CategoryData' })
print(table)


<div class="CategoryData" data-catid="2086"> <ul class="ProductInfoList ProductInfoListBig ListView"> <li data-name="Arduino Uno R3 - clone" data-price="11.5" data-propertys='{"20151":["5.000000000000",null],"20159":["20.000000000000",null],"20184":["7.000000000000","12.000000000000"],"20188":["32000.000000000000",null],"20192":["6.000000000000",null],"20215":["0.020000000000",null],"20216":818,"20217":["16000000.000000000000",null],"20218":["1000.000000000000",null],"20219":["2000.000000000000",null],"20221":["6.000000000000",null]}' data-rating="" itemscope="" itemtype="http://schema.org/Product"><div class="img"><meta content="https://cdn.bodanius.com/media/1/50b119225_arduino-uno-r3-clone_900x600.png" itemprop="image"/><a class="Image" data-ajax="true" data-background="https://cdn.bodanius.com/media/1/50b119225_arduino-uno-r3-clone_x.png" href="/product/arduino-uno-r3-clone" title="Arduino Uno R3 - clone"> </a> <span aria-label="Rating: 4.4 out of 5." class="ReviewStars" style="--r

In [70]:
li_tags = table.findAll('li')


C:\Users\alejo\AppData\Local\Temp\ipykernel_33328\2266616358.py:1: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  li_tags = table.findAll('li')


In [88]:
for i in li_tags: 
    split = i.text.split('\n')
    li_text = split[0].strip(' ')

    print(li_text )

Arduino Uno R3 - clone De Arduino Uno is een bord gebaseerd op de ATmega328 chip. Deze Uno heeft 14 digitale input / output pinnen (waarvan er 6 gebruikt kunnen worden als PWM outputs), 6 analoge inputs. Deze Arduino is erg populair en is door de ingebouwde USB interface erg gemakkelijk te programmeren.   Direct leverbaar   € 11,50  Incl. BTW
Arduino nano I/O shield V3 Deze shield maakt de Arduino Nano een stuk toegankelijker in het gebruik. Doordat deze shield alle I/O van de Arduino netjes groepeert in groepen met GND, SIGNAL en VCC.   Direct leverbaar   € 4,70  Incl. BTW
Arduino pro micro 5V 16MHz - clone De Arduino pro micro is eigenlijk de kleine versie van de Arduino Leonardo , gebaseerd op de ATmega32u4. Het bord bevat door de grootte 12 digitale input/output pinnen (waarvan er 5 gebruikt kunnen worden als PWM outputs), 4 (10 bit) ADC inputs, een 16 MHz crystal en een micro USB connectie.   Direct leverbaar   € 10,50  Incl. BTW
Arduino Nano R3 - kloon - met headers De Arduino Na

In [ ]:
import pandas as pd 

